In [1]:
import pandas as pd
from scipy.optimize import linprog
import numpy as np

In [2]:
df=pd.read_excel('data1.xlsx')
df

,Unnamed: 0,Имя,Цена,Сухое,Натура,Сырой,Крахмал,NDF,ADF,Ca,Phos
0,Имся,единица,руб,кг,кг,кг,кг,кг,кг,кг,кг
1,Ячмень зерно,кг,10.000,0.900,1.111,0.150,0.500,0.200,0.100,0.002,0.003
2,Кукуруза зерно,кг,15.000,0.800,1.250,0.100,0.600,0.250,0.150,0.003,0.004
3,Силос,кг,5.000,0.400,2.500,0.050,0.300,0.400,0.200,0.004,0.005
4,Соя шрот,кг,20.000,0.900,1.111,0.400,0.200,0.300,0.250,0.005,0.006


In [3]:
class method1():
    def __init__(self, nerva_lp, nerva_rp, costs, bnd, ratio=None):
        self.nerva_lp=nerva_lp
        self.nerva_rp=nerva_rp
        self.costs=costs
        self.bnd=bnd
        self.ratio=ratio
#         self.eq_lp=eq_lp
#         self.eq_rp=eq_rp

    def find_opt(self):
        if not self.ratio is None:
            A_ub=np.concatenate([self.nerva_lp, self.ratio], axis=0)
            b_ub=np.concatenate([self.nerva_rp, np.zeros(self.ratio.shape[0])], axis=0)
        else:
            A_ub=self.nerva_lp
            b_ub=self.nerva_rp
                       
        opt = linprog(c=self.costs, 
        A_ub=A_ub,
        b_ub=b_ub,
        #A_eq=eq_lp, b_eq=eq_rp, 
        bounds=self.bnd,
        method="highs")
        return {'success':opt.success, 'x':opt.x, 'fun':opt.fun}
    
    def bnd2condition(comps_bnd):
        cond_l=[]
        cond_r=[]
        for num, i in enumerate(comps_bnd):
            if i[0]:
                cond_l.append(-main_mat[:, num])
                cond_r.append(-i[0])
            if i[1]:
                cond_l.append(main_mat[:, num])
                cond_r.append(i[1])
        cond_l=np.array(cond_l)
        cond_r=np.array(cond_r)
        return cond_l, cond_r
    
    def change_bnd(self):
        #self.bnd=np.stack([np.where(self.bnd[:, 0]-[1/costs[0]]<0, 0, self.bnd[:, 0]-[1/costs[1]]), self.bnd[:, 1]+[1/costs[0]]], axis=1)
        self.nerva_rp=np.where(test1.nerva_rp[:]>0, test1.nerva_rp[:]*1.1, test1.nerva_rp[:]/1.1)
        #print(self.nerva_rp)
        
    def find_rad(self, num):
        counter=0
        if num==1:
            #Постепенно расширяет границы условий
            while not self.find_opt()['success']:
                self.change_bnd()
                print('№', counter, self.nerva_rp)
                counter+=1
        elif num==2:
#             A_ub=self.nerva_lp
#             b_ub=np.where(test1.nerva_rp[:]>0, float('inf'), 0)
            opt = linprog(c=self.costs, 
            A_ub=[[0 , 0]],
            b_ub=[0],
            #A_eq=eq_lp, b_eq=eq_rp, 
            bounds=self.bnd,
            method="highs")
            return {'success':opt.success, 'x':opt.x, 'fun':opt.fun}
            
        return self.find_opt()

In [4]:
datas=df.drop([0], axis=0).drop(['Unnamed: 0', 'Имя'], axis=1).astype(float).to_numpy()
datas

array([[1.000e+01, 9.000e-01, 1.111e+00, 1.500e-01, 5.000e-01, 2.000e-01,
        1.000e-01, 2.000e-03, 3.000e-03],
       [1.500e+01, 8.000e-01, 1.250e+00, 1.000e-01, 6.000e-01, 2.500e-01,
        1.500e-01, 3.000e-03, 4.000e-03],
       [5.000e+00, 4.000e-01, 2.500e+00, 5.000e-02, 3.000e-01, 4.000e-01,
        2.000e-01, 4.000e-03, 5.000e-03],
       [2.000e+01, 9.000e-01, 1.111e+00, 4.000e-01, 2.000e-01, 3.000e-01,
        2.500e-01, 5.000e-03, 6.000e-03]])

In [5]:
costs=datas[:, 0]
costs

array([10., 15.,  5., 20.])

In [6]:
voda=datas[:, 2]
voda

array([1.111, 1.25 , 2.5  , 1.111])

In [7]:
componets_for_div=datas[:, 3:]
componets_for_div

array([[0.15 , 0.5  , 0.2  , 0.1  , 0.002, 0.003],
       [0.1  , 0.6  , 0.25 , 0.15 , 0.003, 0.004],
       [0.05 , 0.3  , 0.4  , 0.2  , 0.004, 0.005],
       [0.4  , 0.2  , 0.3  , 0.25 , 0.005, 0.006]])

In [8]:
divided=np.transpose(np.transpose(componets_for_div)/voda)
divided

array([[0.1350135 , 0.450045  , 0.180018  , 0.090009  , 0.00180018,
        0.00270027],
       [0.08      , 0.48      , 0.2       , 0.12      , 0.0024    ,
        0.0032    ],
       [0.02      , 0.12      , 0.16      , 0.08      , 0.0016    ,
        0.002     ],
       [0.360036  , 0.180018  , 0.270027  , 0.2250225 , 0.00450045,
        0.00540054]])

In [9]:
matrix=np.concatenate([datas[:,1].reshape((4, 1)), divided], axis=1)
matrix

array([[0.9       , 0.1350135 , 0.450045  , 0.180018  , 0.090009  ,
        0.00180018, 0.00270027],
       [0.8       , 0.08      , 0.48      , 0.2       , 0.12      ,
        0.0024    , 0.0032    ],
       [0.4       , 0.02      , 0.12      , 0.16      , 0.08      ,
        0.0016    , 0.002     ],
       [0.9       , 0.360036  , 0.180018  , 0.270027  , 0.2250225 ,
        0.00450045, 0.00540054]])

Первый индекс описывает продукт
1	Ячмень зерно	
2	Кукуруза зерно	
3	Силос	
4	Соя шрот
Второй индекс описывает компонент (кроме натурального, ушёл как коэф.)
1 Сухое
2 Сырой
3 Крахмал
4 NDF
5 ADF
6 Ca
7 Phos

In [10]:
products=[2, 3]#Номера продуктов
comps=[0, 1, 2]#Номера компонентов
comps_bnd=[[2, 4],[1, None],[None, None]]#Ограничения на компоненты
sootnosheniya=[[1, 2, 0], [3, 0, 4]]

Если условие больше, то данные идут с плюсом.
Если меньше, то надо умножить на -1

In [11]:
def get_data():
    main_mat=matrix[products, :][:, comps]
    for_ratio=np.sum(main_mat, axis=1)
    return main_mat, for_ratio

main_mat, for_ratio=get_data()
cond_l, cond_r=method1.bnd2condition(comps_bnd)
print(cond_l)
print()
print(cond_r)

[[-0.4      -0.9     ]
 [ 0.4       0.9     ]
 [-0.02     -0.360036]]

[-2  4 -1]


In [12]:
for_ratio

array([0.54      , 1.44005401])

In [13]:
#Тут лежит соотношение компонента1 к компоненте2 не менее чем 1 к 2
new_line=for_ratio/np.array([-1, 2]) # Тут лежит 1 к 2
# new_line=for_ratio/np.array([-1, 2, float('inf')]) нули так получаются
new_line=new_line.reshape([1, 2])
new_line

array([[-0.54    ,  0.720027]])

In [14]:
test1=method1([[0, 0]], [0], costs[products], bnd=[[10, float('inf')], [2, float('inf')]], ratio=new_line)
opt=test1.find_opt()
opt

{'success': True, 'x': array([10.,  2.]), 'fun': 90.0}

In [15]:
#Проверка соотношения
np.sum(main_mat, axis=1)*opt['x'] # Не менее чем 1 к 2

array([5.4       , 2.88010801])

In [16]:
test1=method1(cond_l, cond_r, costs[products], bnd=[[10, float('inf')], [2, float('inf')]], ratio=new_line)
opt=test1.find_opt()
opt

{'success': False, 'x': None, 'fun': None}

In [17]:
test1.find_rad(1) #Расширить границы

№ 0 [-1.81818182  4.4        -0.90909091]
№ 1 [-1.65289256  4.84       -0.82644628]
№ 2 [-1.5026296  5.324     -0.7513148]
№ 3 [-1.36602691  5.8564     -0.68301346]


{'success': True, 'x': array([10.,  2.]), 'fun': 90.0}

In [18]:
test1.find_rad(2) #Убрать ограничения

{'success': True, 'x': array([10.,  2.]), 'fun': 90.0}

In [19]:
test1=method1(cond_l, cond_r, costs[products], bnd=[[10, float('inf')], [2, float('inf')]])
opt=test1.find_opt()
opt

{'success': False, 'x': None, 'fun': None}